<a href="https://colab.research.google.com/github/ConstanzaG/Colaboratorys/blob/main/apariciones_medios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencias

In [ ]:
# pip install gspread==5.4

In [ ]:
import pandas as pd
import numpy as np

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# cd '/content/drive/Shareddrives/SECOM /Cobertura de medios /Código Apariciones Medios /'

In [ ]:
ruta = '/content/drive/Shareddrives/SECOM /Cobertura de medios /Código Apariciones Medios /'

# Connect to Google Sheets
scope = ['https://www.googleapis.com/auth/spreadsheets',
        #  'https://spreadsheets.google.com/feeds',
         "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name(f"{ruta}secom-google-drive.json", scope)
client = gspread.authorize(credentials)

# Código

## Consolidado de Productos y entregas diarias



In [ ]:
consolidado_ = client.open("Consolidado de Productos y entregas diarias - espejo").sheet1 #Si da error cambiar base por la original
consolidado = pd.DataFrame(consolidado_.get_all_records())

def df_ministerios(consolidado):
  df_final = pd.DataFrame()
  for index, row in consolidado.iterrows():
    ministerios = row['Ministerios'].split(', ')
    df_aux = consolidado.iloc[[index]]
    for m in range(len(ministerios)):
      df_aux['Ministerios'] = ministerios[m]
      df_aux['valor_ministerios'] = 1/len(ministerios)
      df_final = pd.concat([df_final, df_aux], axis=0)
  return(df_final)

#           Guardar Datos
# Abrir la base
sheet = client.open("Consolidado de Productos y entregas diarias - ministerios").sheet1
dfc_f = df_ministerios(consolidado)

# exportar df a excel drive
sheet.clear()
sheet.update( [dfc_f.columns.values.tolist()] + dfc_f.values.tolist(), value_input_option='USER_ENTERED')

{'spreadsheetId': '15HQC-4u2ekYK0_VpcbETp5eDHgrvaMCp7aczyi-I9ho',
 'updatedRange': "'Hoja 1'!A1:O14789",
 'updatedRows': 14789,
 'updatedColumns': 15,
 'updatedCells': 221835}

## Apariciones Medios

In [ ]:
# apariciones_medios_ = pd.ExcelFile("Apariciones Medios.xlsx")
# apariciones_medios = pd.read_excel(apariciones_medios_, 'Base')
# apariciones_medios_ = client.open("Apariciones Medios Base").sheet1 # Base espejo - Error: Resultado muy extenso 
apariciones_medios_ = client.open("Apariciones Medios").sheet1
apariciones_medios = pd.DataFrame(apariciones_medios_.get_all_records())

ponderaciones = {'Television':0.52/8, 'Radio':0.22, 'Periódico':0.14, 'Twitter':0.12}

In [ ]:
# Neutro N:0
# Positivo P:1
# Flanco F:-1
# Riesgo R:1

# Limpiar df
def df_limpio():
    df = apariciones_medios
    df = df.drop(['Positivo', 'Neutro', 'Flanco', 'Riesgo', 'Total', 'Lista', 'Vacios'], axis=1)
    # df = df.dropna(subset=['Temas'])
    df = df.fillna('')
    # df['Prensa digital'] = df['Prensa digital'].replace('', 0)
    df = df[6305:]
    df = df[df['Temas'] != ""]
    return(df)

def valores_tema():
    df_valores = df_limpio()
    col_medios = ['Radio', 'Prensa', 'Prensa digital', 'M- TVN', 'M-CHV', 'M- C13', 'M-Mega', 'Mediodía TVN', 'Mediodía CHV',
            'Mediodía C13', 'MediodíaMega', 'Central TVN', 'Central CHV', 'Central C13', 'Central Mega', 'Twitter']
    tv = ['M- TVN', 'M-CHV', 'M- C13', 'M-Mega', 'Mediodía TVN', 'Mediodía CHV', 'Mediodía C13', 'MediodíaMega', 'Central TVN', 'Central CHV', 'Central C13', 'Central Mega']

    df_valores[col_medios]= df_valores[col_medios].replace({'R':0, 'R ':0, ' R':0,
                                                            'N':0, 'N ':0, ' N':0,
                                                            'P':1, 'P ':1, 'p':1, 'p ':1, ' P':1, ' p':1,
                                                            'F':-1, 'F ':-1, ' F':0,
                                                            '':0, ' ':0, '   ':0})
    # print(1)
    df_valores['Radio'] = df_valores['Radio']*ponderaciones['Radio']
    df_valores['Prensa'] = df_valores['Prensa']*ponderaciones['Periódico']
    # print(2)
    df_valores['Prensa digital'] = df_valores['Prensa digital']*ponderaciones['Periódico']
    # print(3)
    df_valores['Twitter'] = df_valores['Twitter']*ponderaciones['Twitter']
    # print(4)
    df_valores[tv] = df_valores[tv]*ponderaciones['Television']
    # print(5)
    df_valores['valor_tema'] = df_valores[col_medios].sum(axis=1).round(4)
    # print(df_valores[col_medios].values)
    # print(10)
    return(df_valores[col_medios]).sum(axis=1).round(4)

def riesgo():
    df_valores = df_limpio()
    col_medios = ['Radio', 'Prensa', 'Prensa digital', 'M- TVN', 'M-CHV', 'M- C13', 'M-Mega', 'Mediodía TVN', 'Mediodía CHV',
            'Mediodía C13', 'MediodíaMega', 'Central TVN', 'Central CHV', 'Central C13', 'Central Mega', 'Twitter']
    tv = ['M- TVN', 'M-CHV', 'M- C13', 'M-Mega', 'Mediodía TVN', 'Mediodía CHV', 'Mediodía C13', 'MediodíaMega', 'Central TVN', 'Central CHV', 'Central C13', 'Central Mega']

    df_valores[col_medios]= df_valores[col_medios].replace({'R':1, 'R ':1, ' R':1,
                                                            'N':0, 'N ':0, ' N':0,
                                                            'P':0, 'P ':0, 'p':0, 'p ':0, ' P':0, ' p':0,
                                                            'F':0, 'F ':0, ' F':0,
                                                            '':0, ' ':0, '   ':0})
    df_valores['Radio'] = df_valores['Radio']*ponderaciones['Radio']
    df_valores['Prensa'] = df_valores['Prensa']*ponderaciones['Periódico']
    df_valores['Prensa digital'] = df_valores['Prensa digital']*ponderaciones['Periódico']
    df_valores['Twitter'] = df_valores['Twitter']*ponderaciones['Twitter']
    df_valores[tv] = df_valores[tv]*ponderaciones['Television']

    df_valores['Riesgo'] = df_valores[col_medios].sum(axis=1).round(4)
    return(df_valores['Riesgo'] )

def neutro():
    df_valores = df_limpio()
    col_medios = ['Radio', 'Prensa', 'Prensa digital', 'M- TVN', 'M-CHV', 'M- C13', 'M-Mega', 'Mediodía TVN', 'Mediodía CHV',
            'Mediodía C13', 'MediodíaMega', 'Central TVN', 'Central CHV', 'Central C13', 'Central Mega', 'Twitter']
    tv = ['M- TVN', 'M-CHV', 'M- C13', 'M-Mega', 'Mediodía TVN', 'Mediodía CHV', 'Mediodía C13', 'MediodíaMega', 'Central TVN', 'Central CHV', 'Central C13', 'Central Mega']

    df_valores[col_medios]= df_valores[col_medios].replace({'R':0, 'R ':0, ' R':0,
                                                            'N':1, 'N ':1, ' N':1,
                                                            'P':0, 'P ':0, 'p':0, 'p ':0, ' P':0, ' p':0,
                                                            'F':0, 'F ':0, ' F':0,
                                                            '':0, ' ':0, '   ':0})
    df_valores['Radio'] = df_valores['Radio']*ponderaciones['Radio']
    df_valores['Prensa'] = df_valores['Prensa']*ponderaciones['Periódico']
    df_valores['Prensa digital'] = df_valores['Prensa digital']*ponderaciones['Periódico']
    df_valores['Twitter'] = df_valores['Twitter']*ponderaciones['Twitter']
    df_valores[tv] = df_valores[tv]*ponderaciones['Television']

    df_valores['Neutro'] = df_valores[col_medios].sum(axis=1).round(4)
    return(df_valores['Neutro'] )

def Positivos(valor):
    if valor > 0.001:
        valor = valor
    else:
        valor = 0
    return valor

def Flancos(valor):
  if valor < -0.001:
      valor = valor*-1
  else:
      valor = 0
  return valor


In [ ]:
# # Revisar letras de la base

# col_medios = ['Radio', 'Prensa', 'Prensa digital', 'M- TVN', 'M-CHV', 'M- C13', 'M-Mega', 'Mediodía TVN', 'Mediodía CHV',
#             'Mediodía C13', 'MediodíaMega', 'Central TVN', 'Central CHV', 'Central C13', 'Central Mega', 'Twitter']
# for i in col_medios:
#   print(df2[i].unique())


In [ ]:
df2 = df_limpio()
df2.dtypes
df2['Valor']= valores_tema()
df2['Riesgo']= riesgo()
df2['Neutro'] = neutro()
df2['Positivo'] = df2['Valor'].apply(Positivos)
df2['Flanco'] = df2['Valor'].apply(Flancos)

In [ ]:
conditionlist = [
    (df2['Riesgo'] > df2['Positivo']) & ( df2['Riesgo'] > df2['Flanco']) & ( df2['Riesgo'] >= df2['Neutro']),
    (df2['Positivo'] > df2['Riesgo']) & ( df2['Positivo'] > df2['Flanco']) & ( df2['Positivo'] >= df2['Neutro']),
    (df2['Flanco'] > df2['Positivo']) & ( df2['Flanco'] > df2['Riesgo']) & ( df2['Flanco'] >= df2['Neutro']),
    (df2['Neutro'] > df2['Positivo']) & ( df2['Neutro'] > df2['Riesgo']) & ( df2['Neutro'] > df2['Flanco']),
    (df2['Riesgo'] == df2['Positivo']),
    (df2['Riesgo'] == df2['Flanco'])]
    # (df2['Neutro'] == 0) & (df2['Riesgo'] ==0) & (df2['Flanco'] == 0) & (df2['Positivo'] == 0)]
choicelist = ['Riesgo', 'Positivo', 'Flanco', 'Neutro', 'Positivo', 'Flanco']
              #'Neutro']
df2['Valoración'] = np.select(conditionlist, choicelist)

# Guardar datos

In [ ]:
# Abrir la hoja
sheet = client.open("Apariciones Medios Base Ponderada").sheet1

df2['Fecha']= df2['Fecha'].astype(str)
# print (df2.dtypes)

# exportar df a excel drive
sheet.update( [df2.columns.values.tolist()] + df2.values.tolist())

{'spreadsheetId': '1KwrD3Fop45VdG1LaFKppVsWQHiXyhpo1swqHreROXNA',
 'updatedRange': 'Sheet1!A1:AL5831',
 'updatedRows': 5831,
 'updatedColumns': 38,
 'updatedCells': 221578}